<img src="../rsag_convex.png" alt="algoconvex" />
<img src="../x_update.png" alt="x_update" />
<img src="../mean.png" alt="mean" />
<img src="../rsag_composite.png" alt="algo" />

__Parameters :__
- $\alpha$: (1-$\alpha$) weight of aggregated x on current state, i.e. momentum
- $\lambda$: learning rate
- $\beta$: change for aggregated x
- $p_k$ termination probability



In [1]:
from  torch.optim import Adam, SGD, RMSprop
import torch
from torch.nn import functional as F
from torch import nn
import numpy as np

In [2]:
import torch.utils.data as data_utils

In [3]:
print('Using PyTorch version:', torch.__version__)
if torch.cuda.is_available():
    print('Using GPU, device name:', torch.cuda.get_device_name(0))
    device = torch.device('cuda')
else:
    print('No GPU found, using CPU instead.') 
    device = torch.device('cpu')

Using PyTorch version: 2.1.2+cu121
Using GPU, device name: NVIDIA GeForce GTX 1660 Ti


In [4]:
import path
import sys
sys.path.append('../')
from models import MLP
from optimizers import RSAG, AccSGD
from util import DataLoader
from util import calc_accuracy, train_model, HPScheduler


### Run MLP:
__TUNE DIFFERENT OPTIMIZERS__:
- Nesterov w/ weight decay w/ Scheduled LR (SGD)
- Momentum w/ weight decay w/ Scheduled LR (SGD)
- Basic SGD
- Adagrad?
- Adam?



In [5]:
data_loader = DataLoader()
loaders = data_loader.get_loaders()
# loss_function = torch.nn.CrossEntropyLoss()
# model = MLP().to(device)
# print(model)

# optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, nesterov=True, momentum=0.9)
# optimizer = RSAG(model.parameters(), lr=1e-4, alpha=.9, beta=9e-5)


### RSAG Pytorch

In [ ]:
import torch
import warnings
import copy

class RSAG(torch.optim.Optimizer):
    r"""
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate (lambda) (required)
        kappa (float): lambda  (default: 1000)
        xi (float, optional): statistical advantage parameter (default: 10)
        smallConst (float, optional): any value <=1 (default: 0.7)
    Example:
        >>> from RSAG import *
        >>> optimizer = RSAG(model.parameters(), lr=0.1, kappa = 1000.0, xi = 10.0)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    """

    def __init__(self, 
                 params, 
                 lr=0.01, 
                 alpha = 0.1, 
                 beta = 0.1): #, smallConst = 0.7, weight_decay=0):
        #defaults = dict(lr=lr, kappa=kappa, xi, smallConst=smallConst,
                        # weight_decay=weight_decay)
        
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if alpha < 0.0 or alpha > 1.0:
            raise ValueError("Invalid alpha: {}".format(alpha))
        if beta < 0.0:
            raise ValueError("Invalid beta: {}".format(beta))
        
        defaults = dict(lr=lr, alpha=alpha, beta=beta)
        super(RSAG, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RSAG, self).__setstate__(state)

    def step(self, closure=None):
        """ Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            # weight_decay = group['weight_decay']
            lr = group['lr']
            alpha, beta = group['alpha'], group['beta']
            alpha_bar = 1.0-alpha
            momentum_buffer_list = []

            # INITIALIZE GROUPS
            # params_with_grad, d_p_list, momentum_buffer_list = [], [], []
            # for p in group['params']:
            #     if p.grad is not None:
            #         params_with_grad.append(p)
            #         d_p_list.append(p.grad)
            #         # if p.grad.is_sparse:
            #         #     has_sparse_grad = True

            #         state = self.state[p]
            #         if 'momentum_aggr' not in state:
            #             momentum_buffer_list.append(None)
            #         else:
            #             momentum_buffer_list.append(state['momentum_buffer'])
            
            # UPDATE GROUPS
            for p in group['params']:
                if p.grad is None:
                    continue

                d_w = p.grad.data
                # w = p.data
                param_state = self.state[p]

                # if weight_decay != 0:
                #     grad_d.add_(weight_decay, p.data)
                
                if 'momentum_aggr' not in param_state:
                    param_state['momentum_aggr'] = copy.deepcopy(p.data)
                    param_state['prev_momentum_aggr'] = copy.deepcopy(p.data)
                buf = param_state['momentum_aggr']
                aggr_grad = (buf-param_state['prev_momentum_aggr'])
                aggr_grad.mul_(alpha_bar)
                aggr_grad.add_(d_w, alpha=alpha)
                
                param_state['prev_momentum_aggr'] = copy.deepcopy(buf)
                
                # Update momentum buffer:'
                buf.mul_(alpha_bar)
                buf.add_(p.data, alpha=alpha)
                buf.add_(aggr_grad, alpha=-beta)
                
                p.data.add_(aggr_grad, alpha=-lr)
                # print('aggr_grad', aggr_grad)
            
            # UPDATE MOMENTUM BUFFER
            # for p, momentum_buffer in zip(params_with_grad, momentum_buffer_list):
            #     state = self.state[p]
                
            #     state['momentum_buffer'] = momentum_buffer

        return loss

In [ ]:
def tune_nesterov(alpha_values, lr_values, save_log=False):
    loss_function = torch.nn.CrossEntropyLoss()
    best_alpha, best_lr = 0.0, 0.0
    best_accuracy = 0.0
    v_accs, acc_std, v_loss, loss_std = [], [], [], []
    acc, loss = [], []
    
    for alpha in alpha_values:
        for lr in lr_values:
            beta = lr * alpha
            
            print(f"----------- Training with alpha={alpha}, lr={lr} -----------------")
            
            model = MLP().to(device)
            optimizer = SGD(model.parameters(), lr=lr, nesterov=True, momentum=0.9)


            log, best_accuracy = train_model(model,loss_function,optimizer,loaders,print_every=5):
            if log['v_accuracy'][-1] > best_accuracy:
                print(f"Found a new best accuracy: {log['v_accuracy'][-1]}")
                print(f"best alpha: {alpha}, best lr: {lr}")
                best_accuracy = log['v_accuracy'][-1]
                best_alpha = alpha
                best_lr = lr
            
            v_accs.append(log['v_accuracy'])
            acc_std.append(log['v_accuracy_std'])
            v_loss.append(log['v_loss'])
            loss_std.append(log['v_loss_std'])
            acc.append(log['accuracy'])
            loss.append(log['loss'])
            

    
    return best_alpha, best_lr, v_accs, acc_std, v_loss, loss_std, acc, loss


In [6]:
model = MLP().to(device)
print(model)

loss_function = torch.nn.CrossEntropyLoss()


log = train_model(model, loss_function, optimizer, loaders, device)


MLP(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)
Starting Epoch 1


NameError: name 'copy' is not defined

In [81]:
def train_with_hyperparameters(alpha_values, lr_values, save_log=False):
    loss_function = torch.nn.CrossEntropyLoss()
    best_alpha, best_lr = 0.0, 0.0
    best_accuracy = 0.0
    v_accs, acc_std, v_loss, loss_std = [], [], [], []
    acc, loss = [], []
    
    for alpha in alpha_values:
        for lr in lr_values:
            beta = lr * alpha
            
            print(f"----------- Training with alpha={alpha}, lr={lr} -----------------")
            
            model = MLP().to(device)
            optimizer = RSAG(model.parameters(), lr=lr, alpha=alpha, beta=beta)
            log = train_model(model, loaders, optimizer, loss_function, device, epochs=20)
            
            if log['v_accuracy'][-1] > best_accuracy:
                print(f"Found a new best accuracy: {log['v_accuracy'][-1]}")
                print(f"best alpha: {alpha}, best lr: {lr}")
                best_accuracy = log['v_accuracy'][-1]
                best_alpha = alpha
                best_lr = lr
            
            v_accs.append(log['v_accuracy'])
            acc_std.append(log['v_accuracy_std'])
            v_loss.append(log['v_loss'])
            loss_std.append(log['v_loss_std'])
            acc.append(log['accuracy'])
            loss.append(log['loss'])
            

    
    return best_alpha, best_lr, v_accs, acc_std, v_loss, loss_std, acc, loss


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 58)

In [46]:

optimizer = RSAG(model.parameters(), lr=1e-4, alpha=.9, beta=9e-5)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, nesterov=True, momentum=0.9)
train_model(model, loaders, optimizer, loss_function, device, epochs=5)

Starting Epoch 1
Epoch 1 finished
loss 4.6072
Accuracy:  7.4900
Starting Epoch 2
Epoch 2 finished
loss 4.6027
Accuracy:  8.8633
Starting Epoch 3
Epoch 3 finished
loss 4.5982
Accuracy:  10.3017
Starting Epoch 4
Epoch 4 finished
loss 4.5937
Accuracy:  11.6917
Starting Epoch 5
Epoch 5 finished
loss 4.5893
Accuracy:  13.1383
Training has completed
